In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Visualização e utilidades
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

# ==== Funções Fuzzy ====
def trimf(x: float, a: float, b: float, c: float) -> float:
    """
    Função de pertinência triangular.
    """
    if x < a or x > c:
        return 0.0
    elif a <= x <= b:
        return (x - a) / (b - a)
    elif b < x <= c:
        return (c - x) / (c - b)

# ==== Dinâmica do sistema ====
def dinamica(u: np.ndarray, x: float, t: float, T: float) -> tuple:
    """
    Calcula a dinâmica fuzzy superior e inferior.
    """
    tv = np.linspace(0, T, len(u))
    hs = tv[1]
    aux = np.interp(t, tv, hs * np.cumsum(u))
    return x + aux, x - aux

# ==== Construção da solução fuzzy ====
def construir_sol(t: np.ndarray, z: np.ndarray, u: np.ndarray, fuz: tuple) -> np.ndarray:
    """
    Constrói a matriz de solução fuzzy.
    """
    rs = np.zeros((len(z), len(t)))
    u = t[1] * np.cumsum(u)
    for i, vt in enumerate(t):
        for j, vz in enumerate(z):
            rs[j, i] = trimf(vz - u[i], *fuz)
    return rs

# ==== Cálculo dos níveis fuzzy ====
def sol_levels(u: np.ndarray, fuz: tuple, nniv: int) -> list:
    """
    Retorna os limites inferior e superior dos níveis fuzzy.
    """
    niveis = np.linspace(0, 1, nniv)
    xniveis = np.sort(np.concatenate([np.linspace(fuz[0], fuz[2], 10 * nniv), [fuz[1]]])).reshape(-1, 1)
    xpert = np.array([trimf(x[0], *fuz) for x in xniveis]).reshape(-1, 1)
    hs = 1 / len(u)
    sol = hs * np.concatenate([[0], np.cumsum(u)]) + xniveis

    Sup = np.zeros((nniv, len(u) + 1))
    Inf = np.zeros((nniv, len(u) + 1))

    for i in range(nniv):
        candidatos = [sol[j] for j in range(len(xniveis)) if xpert[j] >= niveis[i]]
        if candidatos:
            Sup[i, :] = np.max(candidatos, axis=0)
            Inf[i, :] = np.min(candidatos, axis=0)
        else:
            Sup[i, :] = np.nan
            Inf[i, :] = np.nan

    return [Inf, Sup]

# ==== Funcional objetivo ====
def funcional(u: np.ndarray, gamma: float, beta: float, e: float, x0: float, T: float, alpha: float) -> float:
    """
    Calcula o funcional de custo do controle ótimo fuzzy.
    """
    hs = T / len(u)
    w = hs * np.cumsum(u)
    phit = e * (6 * w**2 - 12 * gamma * w + 12 * x0 * w + e**2 + 6 * gamma**2 - 12 * gamma * x0 + 6 * x0**2) / 6
    Iu2 = hs * np.sum(u**2)
    ju = 0.5 * (1 - alpha) * phit[-1] + 0.5 * alpha * hs * np.sum(phit) + 0.5 * beta * Iu2
    return ju

# ==== Métricas customizadas com PyTorch ====
def minhas_metricas(gamma: float, beta: float, e: float, x0: float, T: float, alpha: float, len_input: int):
    """
    Retorna três funções para usar como métricas ou funções de perda com PyTorch.
    """
    nniv = 100
    xniveis = np.sort(np.concatenate([np.linspace(x0 - e, x0 + e, nniv), [x0]])).reshape(-1, 1)
    xpert = np.array([trimf(x[0], x0 - e, x0, x0 + e) for x in xniveis]).reshape(-1, 1)
    t = np.linspace(0, T, len_input)
    hs = T / len_input

    xniveis_torch = torch.tensor(xniveis.T, dtype=torch.float32)
    xpert_torch = torch.tensor(xpert, dtype=torch.float32)
    dD = torch.tensor(np.abs(xniveis[1:] - xniveis[:-1]), dtype=torch.float32).T
    ap = torch.tensor((xpert[:-1] + xpert[1:]) / 2, dtype=torch.float32).T

    def funcional_metric(y_pred):
        D = hs * torch.cumsum(y_pred, dim=1) + xniveis_torch
        aD = (D[:, :-1] + D[:, 1:]) / 2
        auxD = (aD - gamma) ** 2 * ap
        phit = torch.sum(auxD * dD, dim=1, keepdim=True)
        Iu2 = hs * torch.sum(y_pred ** 2, dim=1, keepdim=True)
        Iphit = hs * torch.sum(phit, dim=1, keepdim=True)
        ju = 0.5 * (1 - alpha) * phit[:, -1] + 0.5 * alpha * Iphit + 0.5 * beta * Iu2
        return torch.mean(ju)

    def loss_u(y_pred):
        t_torch = torch.tensor(t, dtype=torch.float32).unsqueeze(0)
        intu = hs * torch.sum(y_pred, dim=1, keepdim=True)
        aux2 = hs * torch.cumsum(y_pred, dim=1)
        H1 = alpha * e * (T - t_torch) * (gamma - x0)
        H2 = -alpha * e * hs * (torch.sum(aux2, dim=1, keepdim=True) - torch.cumsum(aux2, dim=1))
        H3 = e * (1 - alpha) * (gamma - x0 - intu)
        diff = H1 + H2 + H3 - beta * y_pred
        norma = torch.sqrt(torch.sum(diff ** 2, dim=1)) / len_input
        return torch.mean(norma)

    def funcional2(y_pred):
        w = hs * torch.cumsum(y_pred, dim=1)
        phit = e * (6 * w ** 2 - 12 * gamma * w + 12 * x0 * w + e ** 2 + 6 * gamma ** 2 - 12 * gamma * x0 + 6 * x0 ** 2) / 6
        Iphit = hs * torch.sum(phit, dim=1, keepdim=True)
        Iu2 = hs * torch.sum(y_pred ** 2, dim=1, keepdim=True)
        ju = 0.5 * (1 - alpha) * phit[:, -1] + 0.5 * alpha * Iphit + 0.5 * beta * Iu2
        return torch.mean(ju)

    return funcional_metric, loss_u, funcional2


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/moi6/Github/optimal-control/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/moi6/Github/optimal-control/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/moi6/Github/optimal-control/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739,

In [3]:
import torch
import torch.nn as nn
from test_fuzzy_otimo import minhas_metricas

# ==== Parâmetros ====
gamma = 0.0
beta = 1 / 4.0
e = 1.0
x0 = 5.0
T = 1.0
alpha = 0.5

# Dados da rede
n, n1 = 100, 5

# Dados de treinamento (sem numpy)
X_train_tensor = torch.linspace(0, T, n).unsqueeze(0)
y_train_tensor = torch.linspace(0, T, n).unsqueeze(0)

# Métricas personalizadas
funcional, loss_u, funcional2 = minhas_metricas(gamma, beta, e, x0, T, alpha, n)

# ==== Definição da rede ====
class FuzzyNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FuzzyNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

model = FuzzyNet(n, n1, n)

# Otimizador
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Treinamento
num_epochs = 1001
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train_tensor)
    loss = funcional2(outputs)

    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.6f}")

# Saída final
y_pred = model(X_train_tensor).detach().squeeze().tolist()


Epoch [0/1001], Loss: 12.706872
Epoch [100/1001], Loss: 4.572603
Epoch [200/1001], Loss: 4.563452
Epoch [300/1001], Loss: 4.563452
Epoch [400/1001], Loss: 4.563452
Epoch [500/1001], Loss: 4.563452
Epoch [600/1001], Loss: 4.563452
Epoch [700/1001], Loss: 4.563452
Epoch [800/1001], Loss: 4.563452
Epoch [900/1001], Loss: 4.563452
Epoch [1000/1001], Loss: 4.563452


In [ ]:
# ul = np.load("solutions/din_1_fun_2_epoch11000.npy").ravel()
y_pred = model(X_train_tensor).detach().squeeze().tolist()
t = np.linspace(0,1,len(u)+1)
I,S = sol_levels(u,[x0-e,x0,x0+e],100)
n = len(I)
fig, ax = plt.subplots(figsize=(8, 4))
cmap = plt.get_cmap('bone_r')
for i in range(len(I)):
    fb = ax.fill_between(t, S[i,:], I[i,:], color=cmap(i/n))

ax.set_xlabel(r'$t$', fontsize=20, usetex=True)
ax.set_ylabel(r'$z$', fontsize=20, usetex=True)

ax.set_xlim([0,1])
ax.set_ylim([0,6])

cax, _ = cbar.make_axes(ax,aspect=0.1) 
cb = cbar.ColorbarBase(cax, cmap=plt.cm.bone_r,orientation="horizontal") 
cb.ax.set_position([0.385 ,0.8, 0.1,0.1])
cb.set_ticks([0,0.5,1])

plt.show()

[-7.168189525604248,
 -7.0696258544921875,
 -6.97247314453125,
 -6.876713752746582,
 -6.782326698303223,
 -6.68930196762085,
 -6.597615718841553,
 -6.507246017456055,
 -6.418179988861084,
 -6.330397129058838,
 -6.2438788414001465,
 -6.1586079597473145,
 -6.0745720863342285,
 -5.991750240325928,
 -5.910128116607666,
 -5.829684257507324,
 -5.750410556793213,
 -5.672282695770264,
 -5.595291614532471,
 -5.5194196701049805,
 -5.444652080535889,
 -5.3709716796875,
 -5.298367977142334,
 -5.22681999206543,
 -5.156321048736572,
 -5.086851596832275,
 -5.018399238586426,
 -4.950952529907227,
 -4.884495258331299,
 -4.819013595581055,
 -4.754496097564697,
 -4.6909308433532715,
 -4.628303527832031,
 -4.566600322723389,
 -4.5058112144470215,
 -4.445923328399658,
 -4.386926174163818,
 -4.328802585601807,
 -4.271548748016357,
 -4.215147495269775,
 -4.1595892906188965,
 -4.104862689971924,
 -4.050958633422852,
 -3.997861862182617,
 -3.945568084716797,
 -3.8940603733062744,
 -3.8433332443237305,
 -3.7933

In [ ]:
y